In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from msal import ConfidentialClientApplication
from pyspark.sql import SparkSession
import requests

# Initialize Spark session (if needed)
spark = SparkSession.builder.appName("PowerBI Refresh").getOrCreate()

# Azure AD and Power BI details (replace with your values)
tenant_id = "***YourtenantID***"
client_id = "***YourClientID***"
client_secret = "***YourClientSecret***"
workspace_id = "***YourWorkspaceID***"
dataset_id = "***YourDatasetID***"

# MSAL configuration
authority = f"https://login.microsoftonline.com/{tenant_id}"
scope = ["https://analysis.windows.net/powerbi/api/.default"]

# Gmail SMTP email settings
smtp_server = "smtp.gmail.com"
smtp_port = 587
smtp_user = "***@gmail.com***"  # Your email address
smtp_password = "***YourApplicationPassword***"  # App Password
recipient_email = "***YourRecipientEmail***"  # Recipient's email address

# Function to send email notification
def send_email(subject, body):
    msg = MIMEMultipart()
    msg['From'] = smtp_user
    msg['To'] = recipient_email
    msg['Subject'] = f"Data Refresh Alert - {subject}"
    msg.attach(MIMEText(body, 'plain'))

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Start TLS encryption
            server.login(smtp_user, smtp_password)  # Login with email credentials
            server.sendmail(smtp_user, recipient_email, msg.as_string())
            print("Email sent successfully")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Function to get the access token using MSAL
def get_access_token():
    app = ConfidentialClientApplication(
        client_id,
        authority=authority,
        client_credential=client_secret
    )
    result = app.acquire_token_for_client(scopes=scope)
    if "access_token" in result:
        return result['access_token']
    else:
        raise Exception("Failed to obtain access token")

# Function to trigger dataset refresh
def refresh_dataset(workspace_id, dataset_id, access_token):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    dataset_refresh_url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/datasets/{dataset_id}/refreshes"
    
    refresh_response = requests.post(dataset_refresh_url, headers=headers)
    
    if refresh_response.status_code == 202:
        print("Dataset refresh triggered successfully")
        send_email("Power BI Dataset Refresh Successful", "The dataset refresh was triggered successfully.")
    else:
        print(f"Failed to trigger refresh: {refresh_response.status_code} - {refresh_response.text}")
        send_email("Power BI Dataset Refresh Failed", f"Failed to trigger dataset refresh. Error: {refresh_response.status_code} - {refresh_response.text}")

# Execute the refresh
try:
    access_token = get_access_token()
    refresh_dataset(workspace_id, dataset_id, access_token)
except Exception as e:
    print(f"Error: {e}")
    send_email("Power BI Dataset Refresh Error", f"An error occurred while refreshing the dataset: {e}")



spark.stop()


In [1]:
import os 

print(os.getcwd())

C:\Users\johns
